### Put Call Inequality:
http://moya.bus.miami.edu/~tsu/ijbe2006.pdf
$$C+Ke^{-rT}\le P+S_0\le C+K+S_0(1-e^{-qT})$$
$$\iff S_0e^{-qT}-K\le C-P\le S_0-Ke^{-rT}$$

In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from optionprice import Option
from scipy.optimize import minimize
from datetime import datetime
import QuantLib as ql 

In [2]:
def American_Option(S,K,sigma,r,tau,option_type,q):
    maturity_date = ql.Date(15, 1, 2016)
    calculation_date = maturity_date - tau
    spot_price = S
    strike_price = K
    volatility = sigma # the historical vols or implied vols
    dividend_rate =  q
    risk_free_rate = r
    if option_type == 'Call':
        option_type = ql.Option.Call
    else:
        option_type = ql.Option.Put
        
    day_count = ql.Actual365Fixed()
    calendar = ql.UnitedStates()

    ql.Settings.instance().evaluationDate = calculation_date

    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    settlement = calculation_date

    am_exercise = ql.AmericanExercise(settlement, maturity_date)
    american_option = ql.VanillaOption(payoff, am_exercise)



    spot_handle = ql.QuoteHandle(
        ql.SimpleQuote(spot_price)
    )
    flat_ts = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, risk_free_rate, day_count)
    )
    dividend_yield = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, dividend_rate, day_count)
    )
    flat_vol_ts = ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(calculation_date, calendar, volatility, day_count)
    )
    bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                               dividend_yield, 
                                               flat_ts, 
                                               flat_vol_ts)

    steps = 200
    binomial_engine = ql.BinomialVanillaEngine(bsm_process, "crr", steps)
    american_option.setPricingEngine(binomial_engine)
    return american_option.NPV()

In [3]:
def implied_dividend(S,K,sigma,r,tau,option_type,target_price):
    q_lower = 0
    q_upper = 1
    tol = 1e-2
    diff = 1
    q = 0
    ctr = 0
    while diff > tol:
        q = .5*(q_lower+q_upper)
        option = Option(european=False,kind=option_type,s0=S,k=K,t=tau,sigma=sigma,r=r,dv=q)
        diff = option.getPrice(method = 'MC',iteration = 1000000) - target_price
        if diff < 0:
            q_upper = q
        else:
            q_lower = q
        diff = np.abs(diff)
        ctr += 1
        if ctr > 15:
            return np.NaN
    return q

def implied_dividend_QL(S,K,sigma,r,tau,option_type,target_price):
    q_lower = 0
    q_upper = 1
    tol = 1e-2
    diff = 1
    q = 0
    ctr = 0
    while diff > tol:
        q = .5*(q_lower+q_upper)
        option = American_Option(S,K,sigma,r,tau,option_type,q)
        diff = option - target_price
        if option_type == 'Call':
            if diff < 0:
                q_upper = q
            else:
                q_lower = q
        else:
            if diff > 0:
                q_upper = q
            else:
                q_lower = q
        diff = np.abs(diff)
        ctr += 1
        if ctr > 15:
            return np.NaN
    return q

In [15]:
r = 0.02
df = pd.read_csv("C://Users/ttkad/Dropbox/Summer20/SB/2016con/option500/500option_L2_options_20160630.csv")
df = df.drop(['Unnamed: 0','Exchange','OptionSymbol','OptionExt','OpenInterest','AKA','Delta','Gamma','Theta','Vega'],axis=1)
call = df[df['Type']=='call']
put = df[df['Type']=='put']
df = call.merge(put, left_on=['UnderlyingSymbol','UnderlyingPrice','Expiration','Strike','DataDate'], right_on=['UnderlyingSymbol','UnderlyingPrice','Expiration','Strike','DataDate'])
df = df.drop(['Type_x','Type_y'],axis=1)
new_col_names = ['UnderlyingSymbol','UnderlyingPrice','Expiration','DataDate','Strike','Call_Last','Call_Bid','Call_Ask','Call_Vol','Call_IV','Put_Last','Put_Bid','Put_Ask','Put_Vol','Put_IV']
df = df.rename(columns = {df.columns[i]:new_col_names[i] for i in range(len(df.columns))})
df['Expiration'] = pd.to_datetime(df['Expiration'])
df['DataDate'] = pd.to_datetime(df['DataDate']) 
df['TTM'] = [t.days for t in (df['Expiration'] - df['DataDate'])]
df['risk_free'] = r
df = df[df['Call_Vol']>10]
df = df[df['Put_Vol']>10]
df = df[df['TTM']>10]
df = df.reset_index(drop=True)
df

,UnderlyingSymbol,UnderlyingPrice,Expiration,DataDate,Strike,Call_Last,Call_Bid,Call_Ask,Call_Vol,Call_IV,Put_Last,Put_Bid,Put_Ask,Put_Vol,Put_IV,TTM,risk_free
0,HON,116.32,2016-07-15,2016-06-30,115.0,2.13,2.11,2.36,60,0.1779,1.00,0.79,1.09,121,0.1558,15,0.02
1,HON,116.32,2016-08-19,2016-06-30,110.0,7.52,7.40,7.65,16,0.2278,1.50,1.39,1.45,142,0.2219,50,0.02
2,HON,116.32,2016-08-19,2016-06-30,115.0,3.75,3.75,3.90,54,0.1940,2.83,2.75,2.84,23,0.1941,50,0.02
3,HON,116.32,2016-09-16,2016-06-30,110.0,7.92,7.90,8.10,11,0.2109,2.20,2.01,2.09,125,0.2130,78,0.02
4,HON,116.32,2016-09-16,2016-06-30,115.0,4.35,4.35,4.50,312,0.1848,3.55,3.45,3.60,88,0.1888,78,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,MGM,22.63,2016-07-15,2016-06-30,23.0,0.41,0.40,0.45,32,0.3300,0.83,0.76,0.80,107,0.3170,15,0.02
789,MGM,22.63,2016-07-15,2016-06-30,23.5,0.25,0.22,0.26,86,0.3161,1.13,1.08,1.13,21,0.3073,15,0.02
790,MGM,22.63,2016-08-19,2016-06-30,24.0,0.64,0.61,0.66,24,0.3507,2.20,1.96,2.02,1503,0.3461,50,0.02
791,MGM,22.63,2016-09-16,2016-06-30,22.0,1.87,1.83,1.87,71,0.3707,1.24,1.17,1.20,12,0.3617,78,0.02


In [5]:
call_div = np.array([0.]*len(df.index))
put_div = np.array([0.]*len(df.index))
otm_div = np.array([0.]*len(df.index))
for i in df.index:
    S,K,C,P,sigma_C,sigma_P,r,tau = np.array(df.loc[i,['UnderlyingPrice','Strike','Call_Last','Put_Last','Call_IV','Put_IV','risk_free','TTM']])
    call_div[i] = implied_dividend_QL(S,K,sigma_C,r,tau,'Call',C)
    put_div[i] = implied_dividend_QL(S,K,sigma_P,r,tau,'Put',P)
    if S<K:
        otm_div[i] = call_div[i]
    else:
        otm_div[i] = put_div[i]
df['Call_ID'] = call_div
df['Put_ID'] = put_div
df['Otm_Div'] = otm_div
df

,UnderlyingSymbol,UnderlyingPrice,Expiration,DataDate,Strike,Call_Last,Call_Bid,Call_Ask,Call_Vol,Call_IV,Put_Last,Put_Bid,Put_Ask,Put_Vol,Put_IV,TTM,risk_free,Call_ID,Put_ID,Otm_Div
0,HON,116.32,2016-07-15,2016-06-30,115.0,2.13,2.11,2.36,60,0.1779,1.00,0.79,1.09,121,0.1558,15,0.02,0.148438,0.078125,0.078125
1,HON,116.32,2016-08-19,2016-06-30,110.0,7.52,7.40,7.65,16,0.2278,1.50,1.39,1.45,142,0.2219,50,0.02,0.046875,0.054688,0.054688
2,HON,116.32,2016-08-19,2016-06-30,115.0,3.75,3.75,3.90,54,0.1940,2.83,2.75,2.84,23,0.1941,50,0.02,0.054688,0.041016,0.041016
3,HON,116.32,2016-09-16,2016-06-30,110.0,7.92,7.90,8.10,11,0.2109,2.20,2.01,2.09,125,0.2130,78,0.02,0.042969,0.054688,0.054688
4,HON,116.32,2016-09-16,2016-06-30,115.0,4.35,4.35,4.50,312,0.1848,3.55,3.45,3.60,88,0.1888,78,0.02,0.042969,0.035156,0.035156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,MGM,22.63,2016-07-15,2016-06-30,23.0,0.41,0.40,0.45,32,0.3300,0.83,0.76,0.80,107,0.3170,15,0.02,0.125000,0.093750,0.125000
789,MGM,22.63,2016-07-15,2016-06-30,23.5,0.25,0.22,0.26,86,0.3161,1.13,1.08,1.13,21,0.3073,15,0.02,0.062500,0.062500,0.062500
790,MGM,22.63,2016-08-19,2016-06-30,24.0,0.64,0.61,0.66,24,0.3507,2.20,1.96,2.02,1503,0.3461,50,0.02,0.031250,0.117188,0.031250
791,MGM,22.63,2016-09-16,2016-06-30,22.0,1.87,1.83,1.87,71,0.3707,1.24,1.17,1.20,12,0.3617,78,0.02,0.015625,0.046875,0.046875


In [6]:
df['TTM_N'] = df['Expiration']-df['DataDate']
df['TTM_N'] = [d.days/365 for d in df['TTM_N']]
df['Call_MMP'] =.5*(df['Call_Bid']+df['Call_Ask'])
df['Put_MMP'] =.5*(df['Put_Bid']+df['Put_Ask'])
df['Imp_Div'] =-np.log((df['Call_Last'] - df['Put_Last']+df['Strike'])/df['UnderlyingPrice'])/df['TTM']